In [1]:
import pandas as pd
import numpy as np

# 构建测试数据集
np.random.seed(12)
num_products = 150
advertising_cost = np.random.uniform(500, 5000, num_products)
social_media_expense = np.random.uniform(100, 1000, num_products)
product_price = np.random.uniform(20, 200, num_products)
seasonality_factor = np.random.normal(1, 0.2, num_products)

# 增加两个与销售额无关的变量
employee_satisfaction = np.random.uniform(1, 5, num_products)
monthly_weather_index = np.random.uniform(-10, 10, num_products)

# 生成销售额，考虑以上因素和噪声
sales_revenue = 1000 * advertising_cost + 500 * social_media_expense - 10 * product_price + 200 * seasonality_factor + np.random.normal(0, 5000, num_products)

# 创建数据框
df_sales = pd.DataFrame({
    'Advertising_Cost': advertising_cost,
    'Social_Media_Expense': social_media_expense,
    'Product_Price': product_price,
    'Seasonality_Factor': seasonality_factor,
    'Employee_Satisfaction': employee_satisfaction,
    'Monthly_Weather_Index': monthly_weather_index,
    'Sales_Revenue': sales_revenue
})

# 打印前几行数据
print(df_sales.head())


   Advertising_Cost  Social_Media_Expense  Product_Price  Seasonality_Factor  \
0       1193.732791            679.359714     114.757898            1.055055   
1       3830.223634            693.409016      63.533974            1.105041   
2       1684.917568            234.301317     102.118540            1.079742   
3       2901.827270            771.925411      40.966762            1.438690   
4        565.587331            677.302836     127.965193            0.968143   

   Employee_Satisfaction  Monthly_Weather_Index  Sales_Revenue  
0               2.677083               9.571143   1.539883e+06  
1               3.728390               1.664351   4.170022e+06  
2               3.480114               7.076871   1.793253e+06  
3               3.406356               6.432302   3.291367e+06  
4               3.040455               8.832954   9.026847e+05  


In [2]:
import statsmodels.api as sm
import pandas as pd



# 定义自变量和因变量
X = df_sales[['Advertising_Cost', 'Social_Media_Expense', 'Product_Price', 'Seasonality_Factor', 'Employee_Satisfaction', 'Monthly_Weather_Index']]
y = df_sales['Sales_Revenue']

# 初始化模型，包含常数项
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()

# 打印初始模型的摘要
print("初始模型:")
print(model.summary())

# 逐步回归分析
while True:
    # 获取当前模型的最大p值
    max_pvalue = model.pvalues[1:].idxmax()
    max_pvalue_value = model.pvalues[1:].max()

    # 如果最大p值大于阈值（例如，0.05），则去除该特征
    if max_pvalue_value > 0.05 and max_pvalue != 'const':
        X = X.drop(max_pvalue, axis=1)
        model = sm.OLS(y, X).fit()
        print(f"去除特征 '{max_pvalue}', 当前模型:")
        print(model.summary())
    else:
        break

# 打印最终逐步回归分析的结果
print("最终模型:")
print(model.summary())

初始模型:
                            OLS Regression Results                            
Dep. Variable:          Sales_Revenue   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.289e+06
Date:                Thu, 16 May 2024   Prob (F-statistic):               0.00
Time:                        14:44:06   Log-Likelihood:                -1510.3
No. Observations:                 150   AIC:                             3035.
Df Residuals:                     143   BIC:                             3056.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  -

In [3]:
import statsmodels.api as sm
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def stepwise_regression(X_train, y_train, X_test, y_test, threshold=0.05):
    models_info = []  # 用于存储每个模型的信息
    best_model = None
    best_aic = float('inf')  # 初始化最佳AIC为正无穷
    best_bic = float('inf')  # 初始化最佳BIC为正无穷
    best_features = None
    
    while True:
        # 添加截距项
        X_train = sm.add_constant(X_train)
        X_test = sm.add_constant(X_test)
        
        # 初始化模型
        model = sm.OLS(y_train, X_train).fit()
        models_info.append({
            'Features': X_train.columns[1:],
            'R-squared': model.rsquared,
            'AIC': model.aic,
            'BIC': model.bic,
            'MSE': mean_squared_error(y_test, model.predict(X_test))
        })
        
        # 获取当前模型的AIC和BIC
        current_aic = model.aic
        current_bic = model.bic
        
        # 如果当前模型的AIC或BIC更优，则更新最佳模型和特征
        if current_aic < best_aic and current_bic < best_bic:
            best_aic = current_aic
            best_bic = current_bic
            best_model = model
            best_features = X_train.columns[1:]
        
        # 获取当前模型的最大p值
        max_pvalue = model.pvalues[1:].idxmax()
        max_pvalue_value = model.pvalues[1:].max()

        # 如果最大p值大于阈值，去除该特征
        if max_pvalue_value > threshold:
            X_train = X_train.drop(max_pvalue, axis=1)
            X_test = X_test.drop(max_pvalue, axis=1)
        else:
            break
    
    return {
        'Best_Model': best_model,
        'Best_Features': best_features,
        'Models_Info': pd.DataFrame(models_info)
    }

# 准备数据
X = df_sales[['Advertising_Cost', 'Social_Media_Expense', 'Product_Price', 'Seasonality_Factor', 'Employee_Satisfaction', 'Monthly_Weather_Index']]
y = df_sales['Sales_Revenue']

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 进行逐步回归分析
result = stepwise_regression(X_train, y_train, X_test, y_test)

# 获取最终模型在训练集上的效果
final_train_model = sm.OLS(y_train, sm.add_constant(X_train)).fit()
final_train_r_squared = final_train_model.rsquared
final_train_aic = final_train_model.aic
final_train_bic = final_train_model.bic

# 打印最终模型效果
print("Best Final Model - AIC:", result['Best_Model'].aic, "BIC:", result['Best_Model'].bic)
print("Best Features:", result['Best_Features'])
print("\nFinal Model on Training Set - R-squared:", final_train_r_squared, "AIC:", final_train_aic, "BIC:", final_train_bic)

# 打印模型剔除过程中的关键参数数据
print("\nModels Information:")
print(result['Models_Info'])

Best Final Model - AIC: 2433.998971500511 BIC: 2442.361446728857
Best Features: Index(['Advertising_Cost', 'Social_Media_Expense'], dtype='object')

Final Model on Training Set - R-squared: 0.9999797948530711 AIC: 2439.491175424421 BIC: 2459.003617623895

Models Information:
                                            Features  R-squared          AIC  \
0  Index(['Advertising_Cost', 'Social_Media_Expen...   0.999980  2439.491175   
1  Index(['Advertising_Cost', 'Social_Media_Expen...   0.999980  2437.509276   
2  Index(['Advertising_Cost', 'Social_Media_Expen...   0.999980  2435.972945   
3  Index(['Advertising_Cost', 'Social_Media_Expen...   0.999980  2434.893122   
4  Index(['Advertising_Cost', 'Social_Media_Expen...   0.999979  2433.998972   

           BIC           MSE  
0  2459.003618  2.387636e+07  
1  2454.234227  2.410601e+07  
2  2449.910404  2.477854e+07  
3  2446.043089  2.368623e+07  
4  2442.361447  2.308950e+07  
